In [1]:
#improting all the required packages
import csv
import os
import math
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

In [2]:
#function to check is directory exists
def funCheckDir(filepath):
    directory = os.path.dirname(filepath) # defining directory path
    if not os.path.exists(directory): # checking if directory already exists
        os.makedirs(directory) # making a directory
        return False
    else :
        try:
            os.remove(filepath)
        except OSError:
            pass
        return True

In [3]:
#function to drop blank rows & rows with comments
def funDropRows(df):
    df.dropna(thresh = 2, inplace = True)
    return df

In [4]:
def funDropColumns(df):
    #columns to be selected
    selected_colums = ['loan_amnt', 'term', 'int_rate', 'grade', 'sub_grade', 'emp_title', 'emp_length', 'home_ownership', 
                   'annual_inc', 'verification_status', 'issue_d', 'loan_status', 'purpose', 'addr_state', 'dti', 
                   'earliest_cr_line', 'fico_range_low', 'fico_range_high', 'inq_last_6mths', 'application_type']
    df = df[selected_colums]
    return df

In [5]:
def funCleanData(df):
    #removing string 'months' from term and convertinng into int
    df['term'] = (df['term'].str.extract('(\d+)')).astype(int)
    
    #extracting number from emp_length and convertinng into int
    df['emp_length'] = df['emp_length'].str.extract('(\d+)')
    df['emp_length'] = df['emp_length'].fillna(0).astype(int)
    
    #removing '%' from int_rate and converting into float
    df['int_rate'] = df['int_rate'].apply(lambda x: float(x.rstrip("%")))
    
    return df

In [6]:
def funDeriveColumns(df):
    #deriving a column cr_line_history, to show how old is credit history
    df['earliest_cr_line'] = df['earliest_cr_line'].fillna(df['issue_d'])
    df['cr_line_history'] = df['issue_d'].dt.year - df['earliest_cr_line'].dt.year
    del df['earliest_cr_line']
    
    #deriving issue_month and issue_year from the issue_d column
    df['issue_month'] = df['issue_d'].dt.strftime('%b')
    df['issue_year'] =  df['issue_d'].dt.year
    df['quarter'] =  df['issue_d'].dt.quarter
    df['period'] = df['issue_year'].map(str) + '_Q' +df['quarter'].map(str)
    del df['quarter']
    
    del df['issue_d']
    
    df['fico_score'] = ((df[['fico_range_high', 'fico_range_low']].mean(axis=1)).round()).astype(int)
    del df['fico_range_high']
    del df['fico_range_low']
    
    #'good' loan_status
    good_loan_status_indicators = ['Current','Fully Paid']
    #creating a binary column for the bad Loan Status
    df['loan_status_binary'] = np.where(df['loan_status'].isin(good_loan_status_indicators) , 'Good-Loan', 'Defaulted-Loan')
    
    return df

In [7]:
def funFillMissingData(df):
    #filling NaN with default
    df['home_ownership'] = df['home_ownership'].fillna('ANY')
    df['emp_title'] = df['emp_title'].fillna('Unknown')
    df['purpose'] = df['purpose'].fillna('other')
    df['inq_last_6mths'] = (df['inq_last_6mths'].fillna(0)).astype(int)
    df['loan_status'] = df['loan_status'].fillna('other')
    
    #assuming 'Verified' and 'Verified Source' means the same
    df['verification_status'] = np.where(df['verification_status'] == 'Not Verified' , 'Not Verified', 'Verified')
    
    return df

In [8]:
def funRenameColumns(df):
    #renaming columns
    columns = ['LoanAmt', 'Term', 'IntRate', 'Grade', 'SubGrade', 'EmpTitle', 'EmpLength', 'HomeStatus', 
                   'AnnualInc', 'VerificationStatus', 'LoanStatus', 'Purpose', 'State', 'DTI', 'NoOfInqs', 'AppType',
                   'CreditHistory', 'IssuedMonth', 'IssuedYear', 'Period', 'FICO', 'DefaultStatus']
    df.columns = columns
    
    return df

In [9]:
#function to write the data in chunks
def chunker(seq, size):
    return (seq[pos:pos + size] for pos in range(0, len(seq), size))

In [10]:
#defining the file-directory
fileDir = os.path.dirname(os.path.realpath('__file__'))

In [11]:
columns = ['IssuedYear',  'IssuedMonth', 'LoanAmt', 'Term', 'IntRate', 'Grade', 'SubGrade', 'EmpTitle', 'EmpLength', 
           'HomeStatus', 'AnnualInc', 'VerificationStatus', 'LoanStatus', 'Purpose', 'State', 'DTI', 'NoOfInqs', 
           'AppType', 'CreditHistory', 'FICO', 'DefaultStatus']

In [12]:
#reading loan data stats
for directory, subdirectory, filenames in  os.walk(fileDir + '/data/raw_data/loan_data/'):
    for filename in filenames:
        print("Working on file: " + filename + '....')
        #reading data from CSV
        df = pd.read_csv(os.path.join(directory, filename), skiprows=1, parse_dates=['earliest_cr_line', 'issue_d'])
        df = funDropRows(df)
        df = funDropColumns(df)
        df = funCleanData(df)
        df = funFillMissingData(df)
        df = funDeriveColumns(df)
        df = funRenameColumns(df)
        
        files = df.Period.unique()
        for file in files:
            loanFilePath = fileDir+'/data/processed_data/'+file+'.csv'
            funCheckDir(loanFilePath)
            df_subset = df[df['Period'] == file]
            withHeaders = True
            for i in chunker(df_subset[columns],10000):
                if(withHeaders):
                    i.to_csv(loanFilePath, index=False, mode='a')
                    withHeaders = False
                else:
                    i.to_csv(loanFilePath, index=False, mode='a', header = False)

Working on file: LoanStats3a_securev1.csv....
Working on file: LoanStats3b_securev1.csv....
Working on file: LoanStats3c_securev1.csv....
Working on file: LoanStats3d_securev1.csv....
Working on file: LoanStats_securev1_2016Q1.csv....
Working on file: LoanStats_securev1_2016Q2.csv....
Working on file: LoanStats_securev1_2016Q3.csv....
Working on file: LoanStats_securev1_2016Q4.csv....
